In [2]:
import pandas as pd
from datetime import date

In [3]:
# global recoding on age and pram on sex 
anon_data_global_recoding_pram = pd.read_excel("anon_data/anon_data_global_recoding_pram.xlsx")
# global recoding on age, global recoding on marital status 
anon_data_global2 = pd.read_excel("anon_data/anon_data_global2.xlsx")
# global recodign on age, global recoding on marital status, and pram on sex 
anon_data_global2_pram = pd.read_excel("anon_data/anon_data_global2_pram.xlsx")
# global recodign on age, global recoding on marital status, and pram on sex, local supression 
anon_data_global2_pram_supression = pd.read_excel("anon_data/anon_data_global2_pram_supression.xlsx")

results_data = pd.read_excel("data/public_data_resultsE.xlsx")
pub_data_register = pd.read_excel('data/public_data_registerE.xlsx')

c:\Users\obe\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\obe\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\obe\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


#### Testing if the anonymised datasets can be de-anonymised 

In [4]:
quasi_identifiers1 = ["age_group", "citizenship", "marital_status", "zip"]
quasi_identifiers2 = ["age_group", "citizenship", "marital_status", "zip", "sex"]

def k_anonymity_violations(data, quasi_identifiers, k_levels=[2, 3, 5]):
    # Group by quasi-identifiers to get the counts for each unique combination
    grouped = data.groupby(quasi_identifiers).size().reset_index(name='count')
    
    # Dictionary to store the number of rows violating each k-level
    violations = {}
    total_rows = len(data)

    for k in k_levels:
        # Count rows where the group size is less than k
        violating_rows = grouped[grouped['count'] < k]['count'].sum()
        # Store as a percentage for comparison with R's output
        violations[k] = violating_rows, (violating_rows / total_rows) * 100

    return violations

In [5]:
def identify_k_anonymity_violations(data, quasi_identifiers, k=2):
    # Group by quasi-identifiers to count each unique combination
    grouped = data.groupby(quasi_identifiers).size().reset_index(name='count')
    
    # Filter for combinations where the count is less than k (i.e., violate k-anonymity)
    violating_combinations = grouped[grouped['count'] < k]
    
    # Merge to find the original rows that match these violating combinations
    violations = data.merge(violating_combinations.drop(columns='count'), on=quasi_identifiers, how='inner')
    
    return violations

Global recoding on age, pram on sex

In [6]:
k_anonymity_violations(anon_data_global_recoding_pram, quasi_identifiers1)
vioalting_rows_global1_pram = identify_k_anonymity_violations(anon_data_global_recoding_pram, quasi_identifiers1)

Global recoding on age, global recoding on marital status 

In [7]:
k_anonymity_violations(anon_data_global2, quasi_identifiers1)
vioalting_rows_global2 = identify_k_anonymity_violations(anon_data_global2, quasi_identifiers1)

Global recoding on age, global recoding on marital status, pram on sex

In [8]:
k_anonymity_violations(anon_data_global2_pram, quasi_identifiers1)
vioalting_rows_global2_pram = identify_k_anonymity_violations(anon_data_global2_pram, quasi_identifiers1)

Global recoding on age, global recoding on marital status, pram on sex, local supression 

In [9]:
k_anonymity_violations(anon_data_global2_pram, quasi_identifiers1)


{2: (19, 9.5), 3: (37, 18.5), 5: (84, 42.0)}

#### Deanonymising the dataset 

##### Global recoding on age, global recoding on marital status, pram on sex

Prepare the public register voting data 
- convert dob to age 
- convert age to age groups 
- globally record marital status (like in anonymised data)

In [10]:
# convert dob to age 
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
pub_data_register["dob"] = pd.to_datetime(pub_data_register["dob"])
pub_data_register['age'] = pub_data_register['dob'].apply(lambda x: calculate_age(x))

# convert age to age groups 
age_bins = [18, 30, 38, 48, 58, 70, 101]  # Adjust as needed
age_labels = ["(18,30]", "(30,38]", "(38,48]", "(48,58]", "(58,70]", "(70,101]"] 
pub_data_register['age_group'] = pd.cut(pub_data_register['age'], bins=age_bins,labels= age_labels,right=True)

# define map 
status_map = {
    'Divorced': 'Not married',
    'Married/separated': 'Married',
    'Never married': 'Not married',
    'Widowed': 'Not married'
}
# Apply the map to the marital_status column
pub_data_register["marital_status"]= pub_data_register["marital_status"].map(status_map)
#pub_data_register["marital_status"]= pub_data_register["marital_status"].apply(lambda x: status_map(x))
pub_data_register

,name,sex,dob,zip,citizenship,marital_status,last_voted,age,age_group
0,"Turner, Destiny",Female,2001-04-01,2300,Latvia,Not married,2,23,"(18,30]"
1,"Smith, Ian",Male,1999-07-26,2200,Denmark,Not married,2,25,"(18,30]"
2,"Tucker, Kateland",Female,2001-06-19,2300,Denmark,Not married,1,23,"(18,30]"
3,"Robles Talavera, Troy",Male,2000-02-03,2100,Denmark,Not married,2,24,"(18,30]"
4,"Cordova, Lexis",Female,1999-08-25,2200,Denmark,Not married,2,25,"(18,30]"
...,...,...,...,...,...,...,...,...,...
1558,"Harris, J Shun",Male,1936-03-12,2300,Denmark,Not married,2,88,"(70,101]"
1559,"Todman, Alexis",Female,1968-03-01,2400,Denmark,Not married,0,56,"(48,58]"
1560,"el-Dar, Rushdi",Male,1951-11-07,2400,Denmark,Not married,0,73,"(70,101]"
1561,"el-Salehi, Shaahida",Female,1946-02-14,2200,Denmark,Married,2,78,"(70,101]"


Find the matching rows from the violating rows of the anonymised data i.e. the unqique rows in that dataset and print the quasi-identifiers + the name and the party voted for 

In [11]:
quasi = ['sex', 'marital_status', 'age_group', 'citizenship', "zip"]

with open("data/survey_listE.txt", "r") as my_file:
    # Read the file content
    data = my_file.read()
    
    # Split the text into a list by newline ('\n')
    data_into_list = data.split("\n")

survey_voters = pub_data_register.query('name in @data_into_list')
survey_voters = survey_voters[quasi+["name"]]
survey_voters

,sex,marital_status,age_group,citizenship,zip,name
6,Male,Not married,"(18,30]",Denmark,2100,"al-Ansari, Mukarram"
18,Female,Not married,"(18,30]",Denmark,2100,"Krien, Auji"
24,Female,Not married,"(18,30]",Denmark,2100,"Holman, Rebecca"
31,Male,Not married,"(18,30]",Denmark,2100,"Potter, Andrew"
42,Male,Not married,"(30,38]",Denmark,2100,"Crawleigh, Antonio"
...,...,...,...,...,...,...
1522,Female,Not married,"(48,58]",Denmark,2400,"Gill, Tevis"
1530,Female,Not married,"(70,101]",Denmark,2200,"Siewiyumptewa, Brenna"
1541,Female,Not married,"(38,48]",Denmark,2200,"Wych, Jordyn"
1552,Female,Not married,"(70,101]",Denmark,2200,"Williams, Brianna"


In [12]:
# important columns to select 
quasi_columns = quasi_identifiers1 + ["party", "name"]
# merge the violating rows and the public register dataset 
matched_df = vioalting_rows_global2_pram.merge(survey_voters, on=quasi_identifiers1, how='inner')
# select only relevant columns 
# Count the number of matches for each 'survey_voter_id' (or the unique ID for survey_voters)
match_counts = matched_df['name'].value_counts()

# Filter survey_voters with exactly one match
valid_matches = matched_df[matched_df['name'].isin(match_counts[match_counts == 1].index)]
valid_matches[quasi_columns]

,age_group,citizenship,marital_status,zip,party,name
0,"(30,38]",Germany,Not married,2300,Green,"Liston, Kiet"
1,"(30,38]",Iraq,Married,2200,Green,"al-Morad, Aseela"
2,"(48,58]",Denmark,Married,2100,Green,"Grabb, William"
3,"(38,48]",Turkey,Married,2300,Green,"Baker, Joseph"
4,"(18,30]",USA,Not married,2100,Green,"Brooks, Dakota"
5,"(38,48]",Denmark,Married,2100,Green,"Lopez-Audet, Catelynn"
6,"(70,101]",USA,Married,2400,Green,"Smith, Steele"
7,"(30,38]",Sweden,Not married,2200,Green,"Wing, Heather"
8,"(58,70]",Poland,Married,2400,Green,"Rodriguez-Castorena, Ernesto"
9,"(18,30]",Italy,Not married,2300,Green,"Gutierrez, Sheila"
